# End-to-End Demo

Run a small sweep over physical error rates and report logical failure rates.


In [9]:
import numpy as np
import stim

from quits.circuit import get_qldpc_mem_circuit, check_overlapping_CX
from quits.decoder import (
    sliding_window_bposd_circuit_mem,
    sliding_window_bposd_phenom_mem,
    sliding_window_bplsd_phenom_mem,
)
from quits.qldpc_code import BpcCode, HgpCode, QlpCode
from quits.simulation import get_stim_mem_result


In [10]:
# Common parameters
seed = 1
p = 1e-3
num_rounds = 10
num_trials = 200
W, F = 5, 3
max_iter, osd_order = 10, 1


In [11]:
# Build a small HGP code
h = np.loadtxt(
    "../parity_check_matrices/n=12_dv=3_dc=4_dist=6.txt",
    dtype=int,
)
code = HgpCode(h, h)
code.build_circuit(seed=22)
# For this particular parity check matrix, seed=22 is the one that gives entangling depth 8 (other seeds may give entangling depth 12).
report = code.verify_css_logicals()
print("verify_css_logicals", report)


verify_css_logicals {'css_condition': True, 'lz_commutes_with_X': True, 'lx_commutes_with_Z': True, 'rank_hz': 108, 'rank_hx': 108, 'rank_lz': 9, 'rank_lx': 9, 'k_expected': 9, 'lz_independent_mod_Z_stabilizers': True, 'lx_independent_mod_X_stabilizers': True, 'rank_hz_plus_lz': 117, 'rank_hx_plus_lx': 117, 'dim_ker_hz': 117, 'dim_ker_hx': 117, 'hx_plus_lx_spans_ker_hz': True, 'hz_plus_lz_spans_ker_hx': True, 'pairing_rank': 9, 'pairing_is_identity': True, 'ok': True}


### Outputs/metrics
- `pL`: logical error rate per experiment (any logical flip)
- `LFR`: logical failure rate per round, `1 - (1 - pL)^(1/num_rounds)`


In [12]:
# Small sweep over p
ps = [5e-4, 1e-3, 2e-3]
results = []
for p in ps:
    circuit = stim.Circuit(
        get_qldpc_mem_circuit(code, p, p, p, p, num_rounds, basis="Z")
    )
    detection_events, observable_flips = get_stim_mem_result(
        circuit, num_trials, seed=seed
    )
    logical_pred = sliding_window_bposd_circuit_mem(
        detection_events,
        circuit,
        code.hz,
        code.lz,
        W,
        F,
        max_iter=max_iter,
        osd_order=osd_order,
        tqdm_on=True,
    )
    pL = np.mean((observable_flips - logical_pred).any(axis=1))
    lfr = 1 - (1 - pL) ** (1 / num_rounds)
    results.append((p, float(lfr)))
print(results)


100%|██████████| 200/200 [00:59<00:00,  3.34it/s]

[(0.0005, 0.0), (0.001, 0.002528578544461757), (0.002, 0.009386734904589189)]
